<a href="https://colab.research.google.com/github/gogimandu1019/bigdata202406/blob/main/type2_2nd(blank).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 작업형2
- 빅데이터 분석기사 2회 실기
- 데이터 다운로드 : https://www.kaggle.com/prachi13/customer-analytics
- 데이터 파일 명 : Train.csv (파일명이 다를 경우 파일명을 수정해주세요)
- (데이터가 삭제될 시 수업노트에 유사 데이터 셋을 올려두겠습니다)


## 제품 배송 시간에 맞춰 배송되었는지 예측모델 만들기
- 제공된 데이터
    - X_train.csv
    - y_train.csv
    - X_test.csv

- 학습용 데이터 (X_train, y_train)을 이용하여 배송 예측 모형을 만든 후, 이를 평가용 데이터(X_test)에 적용하여 얻은 예측(시간에 맞춰 도착하지 않을 확률)값을 다음과 같은 형식의 CSV파일로 생성하시오(제출한 모델의 성능은 ROC-AUC 평가지표에 따라 채점)
- 0 정시 도착, 1 정시 도착하지 않음

```
ID,Reached.on.Time_Y.N
3500,0.267
3501,0.578
3502,0.123

```

- 출력을 원하실 경우 print() 함수 활용
- 예시) print(df.head())
- getcwd(), chdir() 등 작업 폴더 설정 불필요
- 파일 경로 상 내부 드라이브 경로(C: 등) 접근 불가

- 데이터 파일 읽기 예제

```
# 데이터 불러오기
import pandas as pd
X_test = pd.read_csv("data/X_test.csv")
X_train = pd.read_csv("data/X_train.csv")
y_train = pd.read_csv("data/y_train.csv")
```

- csv파일 생성 예시

```
pd.DataFrame({'ID': X_test['cust_id'], 'Reached.on.Time_Y.N': pred}).to_csv('003000000.csv', index=False)
```


In [1]:
# 시험환경 세팅 3개 파일 생성 (코드 변경 X)
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

def exam_data_load(df, target, id_name="", null_name=""):
    if id_name == "":
        df = df.reset_index().rename(columns={"index": "id"})
        id_name = 'id'
    else:
        id_name = id_name

    if null_name != "":
        df[df == null_name] = np.nan

    X_train, X_test = train_test_split(df, test_size=0.2, random_state=2022)
    y_train = X_train[[id_name, target]]
    X_train = X_train.drop(columns=[target])

    y_test = X_test[[id_name, target]]
    X_test = X_test.drop(columns=[target])
    return X_train, X_test, y_train, y_test

df = pd.read_csv("Train.csv") # 파일명이 다를 경우 파일명을 수정해주세요
X_train, X_test, y_train, y_test = exam_data_load(df, target='Reached.on.Time_Y.N', id_name='ID')
X_train.to_csv("X_train.csv", index=False)
y_train.to_csv("y_train.csv", index=False)
X_test.to_csv("X_test.csv", index=False)

In [2]:
# 데이터 불러오기
import pandas as pd
X_test = pd.read_csv("X_test.csv")
X_train = pd.read_csv("X_train.csv")
y_train = pd.read_csv("y_train.csv")

X_train.shape, y_train.shape, X_test.shape

((8799, 11), (8799, 2), (2200, 11))

## EDA

In [3]:
# 데이터 확인(X_train)
X_train.head()

,ID,Warehouse_block,Mode_of_Shipment,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Product_importance,Gender,Discount_offered,Weight_in_gms
0,8440,B,Ship,4,1,284,4,low,F,4,4795
1,10849,D,Ship,3,2,201,4,medium,M,10,5565
2,9478,B,Ship,4,4,181,3,medium,F,9,4425
3,2968,B,Ship,6,2,236,4,low,F,61,3140
4,6964,B,Road,5,1,282,4,low,F,9,4766


In [4]:
# 데이터 확인(y_train)
y_train.head()

,ID,Reached.on.Time_Y.N
0,8440,1
1,10849,0
2,9478,1
3,2968,1
4,6964,0


In [6]:
# 결측치 확인 train
X_train.isnull().sum()

ID                     0
Warehouse_block        0
Mode_of_Shipment       0
Customer_care_calls    0
Customer_rating        0
Cost_of_the_Product    0
Prior_purchases        0
Product_importance     0
Gender                 0
Discount_offered       0
Weight_in_gms          0
dtype: int64

In [8]:
# 결측치 확인 test
X_test.isnull().sum()

ID                     0
Warehouse_block        0
Mode_of_Shipment       0
Customer_care_calls    0
Customer_rating        0
Cost_of_the_Product    0
Prior_purchases        0
Product_importance     0
Gender                 0
Discount_offered       0
Weight_in_gms          0
dtype: int64

In [9]:
# 타겟(레이블) 확인 0 정시 도착, 1 정시 도착하지 않음
y_train['Reached.on.Time_Y.N'].value_counts()

Reached.on.Time_Y.N
1    5236
0    3563
Name: count, dtype: int64

In [11]:
# 데이터 타입 확인
X_train.info()
X_test.info()
y_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8799 entries, 0 to 8798
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   ID                   8799 non-null   int64 
 1   Warehouse_block      8799 non-null   object
 2   Mode_of_Shipment     8799 non-null   object
 3   Customer_care_calls  8799 non-null   int64 
 4   Customer_rating      8799 non-null   int64 
 5   Cost_of_the_Product  8799 non-null   int64 
 6   Prior_purchases      8799 non-null   int64 
 7   Product_importance   8799 non-null   object
 8   Gender               8799 non-null   object
 9   Discount_offered     8799 non-null   int64 
 10  Weight_in_gms        8799 non-null   int64 
dtypes: int64(7), object(4)
memory usage: 756.3+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2200 entries, 0 to 2199
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 

In [13]:
# 기초 통계 획인
X_train.describe()
X_test.describe()

,ID,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Discount_offered,Weight_in_gms
count,2200.000000,2200.000000,2200.000000,2200.000000,2200.000000,2200.000000,2200.000000
mean,5441.220455,4.066818,2.985455,211.782727,3.590455,13.693182,3637.502727
std,3176.022751,1.140669,1.420556,47.543896,1.540421,16.570380,1626.351431
min,5.000000,2.000000,1.000000,96.000000,2.000000,1.000000,1005.000000
25%,2754.500000,3.000000,2.000000,172.000000,3.000000,4.000000,1851.000000
50%,5394.500000,4.000000,3.000000,217.000000,3.000000,7.000000,4134.500000
75%,8238.000000,5.000000,4.000000,253.000000,4.000000,10.000000,5033.250000
max,10995.000000,7.000000,5.000000,310.000000,10.000000,65.000000,7588.000000


In [14]:
# 기초 통계 획인 object
X_train.describe(include = "O")

,Warehouse_block,Mode_of_Shipment,Product_importance,Gender
count,8799,8799,8799,8799
unique,5,3,3,2
top,F,Ship,low,F
freq,2929,5972,4225,4450


In [15]:
# 기초 통계 획인 object
X_test.describe(include = "O")

,Warehouse_block,Mode_of_Shipment,Product_importance,Gender
count,2200,2200,2200,2200
unique,5,3,3,2
top,F,Ship,low,M
freq,737,1490,1072,1105


In [18]:
# [Tip] object 타입 고유값 개수 확인
X_train[['Warehouse_block','Mode_of_Shipment', 'Product_importance', 'Gender']].nunique()

Warehouse_block       5
Mode_of_Shipment      3
Product_importance    3
Gender                2
dtype: int64

In [19]:
# X데이터와 y데이터 합치기
df = pd.concat([X_train, y_train['Reached.on.Time_Y.N']], axis = 1)
df.head()

,ID,Warehouse_block,Mode_of_Shipment,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Product_importance,Gender,Discount_offered,Weight_in_gms,Reached.on.Time_Y.N
0,8440,B,Ship,4,1,284,4,low,F,4,4795,1
1,10849,D,Ship,3,2,201,4,medium,M,10,5565,0
2,9478,B,Ship,4,4,181,3,medium,F,9,4425,1
3,2968,B,Ship,6,2,236,4,low,F,61,3140,1
4,6964,B,Road,5,1,282,4,low,F,9,4766,0


In [25]:
# idea? 할인율이 도착에 영향을 미칠까?
cond = df['Discount_offered'] > 5
sum(cond)

5480

In [26]:
sum(df[cond]['Reached.on.Time_Y.N'])

3673

## 데이터 전처리 및 피처엔지니어링

In [29]:
# 라벨 인코딩
from sklearn.preprocessing import LabelEncoder
cols = X_train.select_dtypes(include = 'object').columns

for col in cols:
  le = LabelEncoder()
  X_train[col] = le.fit_transform(X_train[col])
  X_test[col] = le.transform(X_test[col])

X_train.head()

,ID,Warehouse_block,Mode_of_Shipment,Customer_care_calls,Customer_rating,Cost_of_the_Product,Prior_purchases,Product_importance,Gender,Discount_offered,Weight_in_gms
0,8440,1,2,4,1,284,4,1,0,4,4795
1,10849,3,2,3,2,201,4,2,1,10,5565
2,9478,1,2,4,4,181,3,2,0,9,4425
3,2968,1,2,6,2,236,4,1,0,61,3140
4,6964,1,1,5,1,282,4,1,0,9,4766


In [30]:
# trainID 삭제, testID 값만 옮겨둠
X_train = X_train.drop('ID', axis = 1)
X_test_id = X_test.pop("ID")

## 모델 및 평가

In [31]:
# 라이브러리 불러오기
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

In [33]:
# 검증 데이터 분리
x_tr, x_val, y_tr, y_val = train_test_split(X_train, y_train['Reached.on.Time_Y.N'], test_size = 0.2, random_state = 0)
x_tr.shape, x_val.shape, y_tr.shape, y_val.shape

((7039, 10), (1760, 10), (7039,), (1760,))

In [40]:
# 로지스틱 회귀
model = LogisticRegression(random_state = 0)
model.fit(x_tr, y_tr)
pred = model.predict_proba(x_val)
print(roc_auc_score(y_val, pred[:, 1]))

0.7277344064386319


In [41]:
# 의사결정나무
model = DecisionTreeClassifier(random_state = 0)
model.fit(x_tr, y_tr)
pred = model.predict_proba(x_val)
print(roc_auc_score(y_val, pred[:, 1]))

0.648336686787391


In [44]:
# 랜덤포레스트
model = RandomForestClassifier(random_state = 0)
model.fit(x_tr, y_tr)
pred = model.predict_proba(x_val)
print(roc_auc_score(y_val, pred[:, 1]))

0.7605499664654595


In [43]:
# XGBOOST
model = XGBClassifier(random_state = 0)
model.fit(x_tr, y_tr)
pred = model.predict_proba(x_val)
print(roc_auc_score(y_val, pred[:, 1]))

0.7570476190476191


## 예측 및 제출

In [46]:
# 예측
pred= model.predict_proba(X_test)
pred

array([[0.65, 0.35],
       [0.62, 0.38],
       [0.68, 0.32],
       ...,
       [0.6 , 0.4 ],
       [0.64, 0.36],
       [0.42, 0.58]])

In [50]:
# 데이터프레임 만들기
submit = pd.DataFrame({'ID' : X_test_id, 'Reached.on.Time_Y.N' : pred[:,1]}).to_csv("00000.csv", index= False)
submit

In [51]:
# 데이터 확인
pd.read_csv("00000.csv")

,ID,Reached.on.Time_Y.N
0,8285,0.35
1,10192,0.38
2,8675,0.32
3,5753,0.45
4,448,0.97
...,...,...
2195,7193,0.28
2196,4445,0.46
2197,7390,0.40
2198,9977,0.36


In [ ]:
# CSV파일 만들기


##  제대로 제출 되었는지 확인

## 체점
- 수험자는 알 수 없는 부분임

In [ ]:
# you = pd.read_csv("수험번호.csv")
# print("당신의 score는?:",roc_auc_score(y_test['Reached.on.Time_Y.N'], you['Reached.on.Time_Y.N']))

In [ ]:
# 당신의 score는?: 0.7396611568179091